In [10]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.utils import save_model, load_model
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease

In [11]:
df = pd.read_csv("data/train.csv", index_col="ID")
df.head()

,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit
ID,,,,,,,,
250,321,111,3,3.5,4.0,8.83,1,0.77
434,316,111,4,4.0,5.0,8.54,0,0.71
20,303,102,3,3.5,3.0,8.50,0,0.62
323,314,107,2,2.5,4.0,8.27,0,0.72
333,308,106,3,3.5,2.5,8.21,1,0.75


In [12]:
X = df.drop(columns="Chance_of_Admit")
y = df.Chance_of_Admit

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((320, 7), (80, 7), (320,), (80,))

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from jcopml.tuning import random_search_params as rsp

In [14]:
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(), ["GRE", "TOEFL", "CGPA", "SOP", "LOR"]),
    ('categoric', cat_pipe(encoder='onehot'), ["University_Rating", "Research"]),
])


pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', RandomForestRegressor(n_jobs=-1, random_state=42))
])

model = RandomizedSearchCV(pipeline, rsp.rf_params, cv=5, n_iter=50, n_jobs=-1, verbose=1, random_state=42)
model.fit(X_train, y_train)

print(model.best_params_)
print(model.score(X_train, y_train), model.best_score_, model.score(X_test, y_test))

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   23.8s finished


{'algo__max_depth': 46, 'algo__max_features': 0.4756699028339012, 'algo__min_samples_leaf': 5, 'algo__n_estimators': 196}
0.8744720634282263 0.792250540761407 0.8181838000996408


In [15]:
def submit(model, filename="wira__v1.csv"):
    df_submit = pd.read_csv("data/test.csv", index_col="ID")
    df_submit['Chance_of_Admit'] = model.predict(df_submit)
    df_submit[['Chance_of_Admit']].to_csv(filename, index_label='ID')

In [16]:
submit(model)